# 📓 02_team_simulation.ipynb
**Purpose:** Simulate Day 2 team scores based on player sampling and calculate win probabilities.

In [ ]:
import random
import numpy as np
import pickle
import matplotlib.pyplot as plt

In [ ]:
# --- Load Data ---

def load_data(filename='golf_data.pkl'):
    with open(filename, 'rb') as f:
        data = pickle.load(f)
    return data['players'], data['tournaments']

PICKLE_FILE = 'path_to_your_saved_pickle.pkl'  # TODO: Update this!
players, tournaments = load_data(PICKLE_FILE)
print(f"✅ Loaded {len(players)} players and {len(tournaments)} tournaments.")

In [ ]:
# --- Sampling Functions ---

def get_net_scores(player):
    return [r.net for r in player.rounds]

def sample_score_simple(player):
    net_scores = get_net_scores(player)
    return random.choice(net_scores) if net_scores else None

def sample_score_weighted(player, decay_factor=0.9):
    net_scores = get_net_scores(player)
    n = len(net_scores)
    if n == 0:
        return None
    weights = [decay_factor ** (n - i - 1) for i in range(n)]
    return random.choices(net_scores, weights=weights, k=1)[0]

In [ ]:
# --- Simulate Single Team Score ---

def simulate_team_score(team, sampling_fn=sample_score_simple):
    """
    Given a Team object, simulate one Day 2 total net score.
    Sampling function can be simple or weighted.
    """
    player_scores = []
    for player in team.members:
        sampled_score = sampling_fn(player)
        if sampled_score is not None:
            player_scores.append(sampled_score)

    if len(player_scores) < 4:
        # Not enough valid scores, team cannot score (return high number)
        return 999

    # Best 4 of 6 scores count
    best_4_scores = sorted(player_scores)[:4]
    return sum(best_4_scores)

In [ ]:
# --- Simulate Entire Tournament ---

def simulate_tournament(teams, num_simulations=10000, sampling_fn=sample_score_simple):
    """
    Simulate many tournaments and estimate win probabilities for each team.
    """
    team_wins = {team.name: 0 for team in teams}

    for _ in range(num_simulations):
        team_totals = {team.name: simulate_team_score(team, sampling_fn) for team in teams}
        winning_team = min(team_totals, key=team_totals.get)
        team_wins[winning_team] += 1

    # Calculate win probabilities
    team_win_probs = {team: wins / num_simulations for team, wins in team_wins.items()}
    return team_win_probs

In [ ]:
# --- Example Setup ---

# TODO: Load your actual teams here.
# Example dummy setup for now

class Team:
    def __init__(self, name, members):
        self.name = name
        self.members = members

# Example usage (update this with real tournament teams):
example_teams = [
    Team("Team A", random.sample(list(players.values()), 6)),
    Team("Team B", random.sample(list(players.values()), 6))
]

team_win_probs = simulate_tournament(example_teams, num_simulations=1000)
print("\nWin Probabilities:")
for team, prob in team_win_probs.items():
    print(f"{team}: {prob:.2%}")